# Bachelier Embedding - Vanilla Call

- Call payoff g(X_T) = (x_T-K)^+

- $p_{BS} = $ Black-Scholes price of a call option with $x_0 = K = 100$, $\sigma_{BS} = 20\%$, $r = 0$. 

- Bachelier embeddings $f_{\sigma}(X_t) = u_{\sigma}(t,x_t)$, where $u_{\sigma}(t,x) = \mathbb{E}^{\mathbb{Q}_\sigma}[g(X_T) \ | \ x_t = x]  \ \to \ g(X_T)$ as $t\to T$. Note that  $ u_{\sigma}(0,x_0) =: u_{\sigma}^0$ is the Bachelier price. 

- First order Taylor ($\lVert \cdot \rVert = $ weighted length)

$$g(X_T) \approx \sum_{\lVert \alpha \rVert \le 1} \Delta_{\alpha}f_{\sigma_{BA}}(X_0) \mathcal{S}_{\alpha}(X_T) =  u_{\sigma_{BA}}^0 + \partial_x u_{\sigma_{BA}}^0 (x_T - x_0) + \partial_t u_{\sigma_{BA}}^0 T +  \partial_{xx} u_{\sigma_{BA}}^0 \frac{(x_T - x_0)^2}{2}$$

As $u_{\sigma_{BA}}$ solves the heat equation $(\partial_t + \frac{\sigma_{BA}^2}{2}\partial_{xx})u = 0$, we can write

$$g(X_T) \approx   u_{\sigma_{BA}}^0 + \partial_x u_{\sigma_{BA}}^0 (x_T - x_0)  +  \partial_{xx} u_{\sigma_{BA}}^0 \frac{(x_T - x_0)^2 - \sigma_{BA}^2 T}{2}$$

[ In the Bachelier($\sigma_{BA}$) model, note that   $\frac{(x_T - x_0)^2 - \sigma_{BA}^2 T}{2} =  \int_0^T\int_0^{t_2} dx_{t_1}dx_{t_2} =  \sigma_{BA}^2 \int_0^T\int_0^{t_2} dw_{t_1}dw_{t_2}$ (2nd order Hermite polynomial) ]

Taking expectation w.r.t. to $\mathbb{Q}$ (Black-Scholes model) gives
$$\text{price}_{BS} \approx  u_{\sigma_{BA}}^0 + \partial_{xx} u_{\sigma_{BA}}^0 \frac{e^{\sigma_{BS}^2T} - 1 - \sigma_{BA}^2 T}{2} \approx \text{price}_{BA} + \frac{1}{2}\Gamma_{\sigma_{BA}}^{BA} T(\sigma_{BS}^2- \sigma_{BA}^2) $$
It is therefore wise to set $\sigma_{BA}$ close to $\sigma_{BS}$. However, there should be a premium under the Black-Scholes model due to the convexity of $e^{\cdot}$, so the "optimal" value for $\sigma_{BA}$ should be larger than $\sigma_{BS}$. A natural choice is $\sigma_{BA} = \sqrt{\mathbb{V}^{\mathbb{Q}}[x_1]} = \sqrt{\frac{e^{\sigma_{BS}^2T} - 1}{T}} > \sigma_{BS}$ , although close to $\sigma_{BS}$ itself.

If $x_0 = K$, it turns out that $$\text{price}_{BS} \le u_{\sigma_{BA}}^0 + \frac{T}{2}\partial_{xx} u_{\sigma_{BA}}^0 (\mathbb{V}^{\mathbb{Q}}[x_1] - \sigma_{BA}^2),$$
where the infimum on the RHS seems to be  attained at  $\sigma_{BA} = \sqrt{\mathbb{V}^{\mathbb{Q}}[x_1]} = \sqrt{\frac{e^{\sigma_{BS}^2T} - 1}{T}}$. 

In [1]:
import numpy as np
from scipy.stats import norm as nm
import bqplot.pyplot as bqplt
import ipywidgets as wi
from scipy.stats import norm
from bqplot import Axis, LinearScale
from numpy import random as rdm
import matplotlib.pyplot as plt; plt.style.use('dark_background')
from cycler import cycler as cyc
__ = np.newaxis

In [2]:
def BM(Ts,nSim):
    dts = np.diff(Ts,axis=0)[:,np.newaxis]# , keepdim = True)
    W   = np.cumsum(np.sqrt(dts) * rdm.randn(len(dts),nSim),axis=0)
    W   = np.vstack((np.zeros((1,nSim)),W))
    return W

def bsVanilla(x0,K,r,delta,sig,T):
    """Black-Scholes price and greeks for vanilla options (call and put)"""
    # Discount factor
    D = np.exp(-r*T)
    # Forward Price
    F = x0/D * np.exp(-delta*T)
    # Volatility over [0,T]
    sigT = sig * np.sqrt(T)
    # Formula when volatility and time to maturity are positive
    if sigT > 0:
        # Thresholds (i = 1,2)
        d = lambda i: np.log(F/K)/sigT - (-1)**i * sigT/2
        # Call Price
        C = D * (F * nm.cdf(d(1)) - K * nm.cdf(d(2)))
        # Delta of Call Option
        dC = np.exp(-delta*T) * nm.cdf(d(1))
        # Gamma of Call/Put Option (same)
        g = np.exp(-delta*T) * nm.pdf(d(1)) / (x0 * sigT)
        # Vega of Call/Put Option (same)
        v = g * x0**2 * sig * T
    elif T == 0: 
        C, dC, g, v = np.maximum(x0 - K,0.), 1 * (x0 >= K), 0.*x0, 0.*x0
    elif sig == 0: 
        C, dC, g, v = D * np.maximum(F - K,0.), np.exp(-delta*T) * (F >= K), 0.*x0, 0.*x0
    # Put Price and Delta (use Put-Call parity!)
    P, dP = C + D * (K - F), dC - 1
    return  C, P , dC, dP, g, v 

# Black-Scholes Parameter (delta is the dividend rate of the underlying. In this class, we always assume that delta = 0)
x0,K,r,delta,sigBS,T = 100,100,0.0,0.0,0.2,0.5
# "Optimal" Bachelier volatility
sigOpt = np.sqrt((np.exp(sigBS**2 * T) - 1)/T)

# Call Payoff 
g = lambda X: np.maximum(X[-1] - K,0.)

def MC(g,T = T, r = 0.):
    return np.mean(np.exp(-r*T) * g,axis = -1)

# Bachelier Embedding
def u(tau,x,K,sigBa): 
    """Bachelier Embedding. Tau = time to maturity"""
    sig_ = sigBa * np.sqrt(np.maximum(tau,1e-10))
    u1   = (x - K) * norm.cdf((x - K)/sig_)
    u2   = sig_ * norm.pdf((x - K)/sig_)
    return np.where(tau > 0., u1 + u2, x - K)

def statRep(sigBa = None,T = T,h = 0.01, dT = 0.001):  
    """Static Replication (order 1). Returns hedged values and price"""
    # Functional derivatives
    if sigBa is None: sigBa = sigBS * x0
    Delta       = {}
    Delta["_"]  = u(T,x0,K,sigBa)  # Bachelier price
    Delta["t"]  = (u(T-dT,x0,K,sigBa) - u(T,x0,K,sigBa))/dT
    Delta["x"]  = (u(T,x0 + h,K,sigBa) - u(T,x0,K,sigBa))/h
    Delta["xx"] = (u(T,x0 + h,K,sigBa) - 2 * u(T,x0,K,sigBa) + u(T,x0 - h,K,sigBa))/h**2
    return Delta


def g1(xT,Delta,T = T):
    return Delta["_"] + Delta["t"] * T + Delta["x"] * (xT - x0) + Delta["xx"] * (xT - x0)**2/2

Let $f_{\sigma}(X_t) = \iota_{\mathbb{Q}_{\sigma}} g(X_t)$

Firts order Taylor: $t\in [0,T]$
$$ g(X_T) \approx f_{\sigma}(X_t) + \Delta_t f_{\sigma}(X_t) (T - t) + \Delta_x f_{\sigma}(X_t) (x_T - x_t)  $$ 


In [3]:
MC(G1)

NameError: name 'G1' is not defined

In [ ]:
N = int(1e3); ts  = np.linspace(0,T,N+1); Ts  = np.array([ts]).T

nSim = 10000; rdm.seed(11)
X    = x0 * np.exp(sigBS * BM(ts,nSim) + (r - sigBS**2/2) * Ts)
G    = g(X)

pBS,*_ = bsVanilla(x0,K,r,delta,sigBS,T)
print("BS price: %2.2f"%pBS)
    
for s in [0.01,0.2,0.5]:
    Delta = statRep(sigBa = s*x0); G1 = g1(X[-1],Delta)
    plt.scatter(G,G1,s = 3,color = "steelblue"); plt.xlabel(r"$g(X_T)$")
    plt.ylabel(r"$\hat{g}_1(X_T)$",rotation = 0)
    plt.plot([np.min(G),np.max(G)],[np.min(G1),np.max(G1)],color = "w")
    plt.title("Bachelier vol = %2.2f. Price = %2.2f"%(s*x0,MC(G1))); plt.show()
    
    xs = np.linspace(0.75*x0,1.25*x0,201); G1 = g1(xs,Delta)
    plt.plot(xs,G1); plt.plot(xs,g(xs[__,:])); 
    plt.title("Bachelier vol = %2.2f"%(s*x0)); plt.show()

## Final Payoff

In [ ]:
# Stock Grid
xMin,xMax = 0.75*K, 1.25*K
xs        = np.linspace(xMin,xMax,1001)
# Max Bachelier volatility (normalized)
sigMin, sigMax = 0.05, 1.0

def update_Price(message):
    if message['name'] == 'value':
        sig_         = slider.value
        Delta        = statRep(sigBa = sig_ * x0)
        plot.y       = g1(xs,Delta)
        figure.title = title(sig_)

#===================== PLOT =====================# 
title = lambda sig: 'Call payoff, K = %2.f, T = %2.f, Bachelier volatility = %2.f%%'%(K,T,100*sig)

figure = bqplt.figure(title = title(sigMin),animation_duration=0,legend_location = "top-left")
bqplt.xlabel("Stock Price"); bqplt.xlabel("Payoff") 
# Call payoff
bqplt.plot(xs,g(xs[__,:]),stroke_width = 3,labels = ["Call Payoff"],
           display_legend = False,colors = ["cornflowerblue"],opacities = [0.8])
# Replication, BS vol
bqplt.plot(xs,g1(xs,statRep(sigBa = sigBS * x0)),stroke_width = 3,
           labels = ["Static Replication (BS Vol)"], display_legend = True, 
           colors = ["crimson"],opacities = [1.0])
# Replication, init vol
plot = bqplt.plot(xs,g1(xs,statRep(sigBa = sigMin * x0)),stroke_width = 3,
                  labels = ["Static Replication"],display_legend = True, colors = ["plum"],
                  axes_options={"x": {"label": "Stock Price", "num_ticks":21}})

plot.observe(update_Price, ['x','y'])
bqplt.xlim(xMin,xMax); bqplt.ylim(-K/20,xMax-K+1)
# Slider (log2 of number of time steps)
slider = wi.FloatSlider(description=r'$\sigma_{\text{Bach}}$', step=0.01, min = sigMin, max = sigMax)
slider.observe(update_Price,'value')
# Display animation
wi.VBox([slider, figure])

## Initial Price

In [4]:
# Stock Grid
xMin,xMax = 0.6*K, 1.4*K; yMin, yMax = -K/20,xMax-K+1
xs     = np.linspace(xMin,xMax,51)
# BS prices
pBS,*_ = bsVanilla(xs,K,r,delta,sigBS,T)
# Max Bachelier volatility (normalized)
sigMin, sigMax = 0.1, 0.4
# BS paths starting from 1
nSim = 1000
X1   = np.exp(sigBS * np.sqrt(T) * rdm.randn(1,nSim) + (r - sigBS**2/2) * T)

def update_Price(message):
    if message['name'] == 'value':
        sig_         = slider.value
        Delta        = statRep(sigBa = sig_ * x0)
        plot.y       = [MC(g1(x * X1,Delta)) for x in xs]
        figure.title = title(sig_)

#===================== PLOT =====================# 
title = lambda sig: 'Call Price, K = %2.f, T = %2.f, Bachelier volatility = %2.f%%'%(K,T,100*sig)

figure = bqplt.figure(title = title(sigMin),animation_duration=0,legend_location = "top-left")
bqplt.xlabel("Stock Price"); bqplt.xlabel("Payoff") 
bqplt.plot([x0,x0],[yMin,yMax],colors = ["gray"],opacities = [0.8])
# Call payoff
bqplt.plot(xs,pBS,stroke_width = 3,labels = ["Black-Scholes"],display_legend = True,colors = ["cornflowerblue"])
# Price, BS vol
Delta = statRep(sigBa = sigBS * x0)
bqplt.plot(xs,[MC(g1(x * X1,Delta)) for x in xs],stroke_width = 3,
           labels = ["Static Replication (BS Vol)"], display_legend = True, 
           colors = ["plum"],opacities = [0.8],zorders = [9])
# Price, init vol
Delta = statRep(sigBa = sigMin * x0)
plot = bqplt.plot(xs,[MC(g1(x * X1,Delta)) for x in xs],stroke_width = 3,
                  labels = ["Static Replication"],display_legend = True, colors = ["darkred"],
                  axes_options={"x": {"label": "Stock Price", "num_ticks":21}})

plot.observe(update_Price, ['x','y'])
bqplt.xlim(xMin,xMax); bqplt.ylim(yMin, yMax)
# Slider (log2 of number of time steps)
slider = wi.FloatSlider(description=r'$\sigma_{\text{Bach}}$', step=0.01, min = sigMin, max = sigMax)
slider.observe(update_Price,'value')
# Display animation
wi.VBox([slider, figure])

## Term Structure

In [8]:
# Stock Grid
TMin,TMax = 0., 0.5
Ts        = np.linspace(TMin,TMax,101); Ts_ = np.array([Ts]).T
# BS prices
pBS = [bsVanilla(x0,K,r,delta,sigBS,T_)[0] for T_ in Ts]
# Max Bachelier volatility (normalized)
sigMin, sigMax = 0.05, 0.41
# BS paths
nSim = int(1e5)
X    = x0 * np.exp(sigBS * np.sqrt(Ts_) * rdm.randn(len(Ts),nSim) + (r - sigBS**2/2) * Ts_)
  
def update_Price(message):
    if message['name'] == 'value':
        sig_           = slider.value
        Delta          = statRep(sigBa = sig_ * x0,T = Ts_)
        plot.y         = MC(g1(X,Delta,Ts_),Ts_) 
        figure.title   = title(sig_)

#===================== PLOT =====================# 
title = lambda sig: 'ATM Call Term Structure, Bachelier volatility = %2.f%%'%(100*sig)

figure = bqplt.figure(title = title(sigMin),animation_duration=0,legend_location = "top-left")
bqplt.xlabel("Stock Price"); bqplt.xlabel("Payoff") 
# BS price
bqplt.plot(Ts,pBS,stroke_width = 2,labels = ["Black-Scholes"],display_legend = True,colors = ["cornflowerblue"])
# Price, BS vol
Delta = statRep(sigBa = sigBS * x0,T = Ts_)
bqplt.plot(Ts,MC(g1(X,Delta,Ts_),Ts_),stroke_width = 2,
           labels = ["Static Replication (BS Vol)"], display_legend = True, 
           colors = ["plum"],opacities = [0.8],zorders = [9])
# "Optimal" sigma

Delta = statRep(sigBa = sigBS * x0,T = Ts_)
bqplt.plot(Ts,MC(g1(X,Delta,Ts_),Ts_),stroke_width = 2,
           labels = ["Static Replication (BS Vol)"], display_legend = True, 
           colors = ["plum"],opacities = [0.8],zorders = [9])
# Price, init vol
Delta = statRep(sigBa = sigMin * x0,T = Ts_)
plot = bqplt.plot(Ts,MC(g1(X,Delta,Ts_),Ts_),stroke_width = 2,
                  labels = ["Static Replication"],display_legend = True, colors = ["darkred"],
                  axes_options={"x": {"label": "Stock Price", "num_ticks":21}})

plot.observe(update_Price, ['x','y'])
bqplt.xlim(TMin,TMax); bqplt.ylim(0.,pBS[-1]*1.2)
# Slider (log2 of number of time steps)
slider = wi.FloatSlider(description=r'$\sigma_{\text{Bach}}$', step=0.01, min = sigMin, max = sigMax)
slider.observe(update_Price,'value')
# Display animation
wi.VBox([slider, figure])

In [229]:
g1

<function __main__.<lambda>(xT, Delta)>

In [207]:
MC(g1(X,Delta),Ts_)

array([0.        , 2.43834878, 2.62876136])

In [209]:
Delta


{'_': array([[0.        ],
        [0.9973557 ],
        [1.41047396]]),
 't': array([[ 0.        ],
        [-1.99671011],
        [-1.4111799 ]]),
 'x': array([[1.        ],
        [0.50079788],
        [0.50056419]]),
 'xx': array([[0.        ],
        [0.1595767 ],
        [0.11283784]])}

In [210]:
g1(X,Delta)

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 1.83170175,  0.1902713 , -0.78657092,  8.80225989,  2.15408185],
       [ 7.04030495,  2.36132514,  3.58020019, -0.28797151,  0.44994801]])

In [214]:
Delta["x"] * (X-x0) + Delta["t"] * Ts_ 

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.79811161, -0.31831369, -2.09901617,  3.41928689, -5.11124421],
       [ 2.82426886, -6.43150471, -7.13337314, -2.20399486, -0.97712821]])

In [202]:
X

(51, 10000)

In [193]:
def statRep(sigBa = None,T = T,h = 0.01, dT = 0.001):  
    """Static Replication (order 1). Returns hedged values and price"""
    # Functional derivatives
    if sigBa is None: sigBa = sigBS * x0
    Delta       = {}
    Delta["_"]  = u(T,x0,K,sigBa)  # Bachelier price
    Delta["t"]  = (u(np.maximum(T-dT,0.),x0,K,sigBa) - u(T,x0,K,sigBa))/dT
    Delta["x"]  = (u(T,x0 + h,K,sigBa) - u(T,x0,K,sigBa))/h
    Delta["xx"] = (u(T,x0 + h,K,sigBa) - 2 * u(T,x0,K,sigBa) + u(T,x0 - h,K,sigBa))/h**2
    return Delta

Ts = np.linspace(TMin,TMax,5); Ts_ = np.array([Ts]).T
statRep(sigBa = sigBS * x0,T = Ts_)

{'_': array([[0.        ],
        [2.82094792],
        [3.9894228 ],
        [4.88602512],
        [5.64189584]]),
 't': array([[  0.        ],
        [-11.30644998],
        [ -7.98684045],
        [ -6.51904909],
        [ -5.64471961]]),
 'x': array([[1.        ],
        [0.50028209],
        [0.50019947],
        [0.50016287],
        [0.50014105]]),
 'xx': array([[0.        ],
        [0.05641895],
        [0.03989422],
        [0.0325735 ],
        [0.02820948]])}

In [119]:
MC(g1(X,Delta),T = Ts_)

2.632125039605402

In [112]:
X

array([[102.30382613,  98.97119933,  96.60414617],
       [110.53589391,  92.46565784,  93.84023349],
       [108.39693476, 101.60545788,  96.89701829],
       [ 94.88058912, 114.18923714, 101.61931622],
       [ 80.47183911,  97.6408856 , 107.82729746],
       [109.99473323, 105.6227037 ,  92.01318055],
       [ 85.56341287, 109.57151099, 111.80071234],
       [113.82245589,  70.79663284,  92.54964215],
       [138.11597939,  94.57814856,  89.12979796],
       [111.74478115,  67.64085786, 101.24168833],
       [ 62.86206895,  90.13011052,  84.54134818]])

In [114]:
np.exp(-r*Ts_) * g1(X,Delta)

array([[ 4.03380088,  2.30708379,  1.27094781],
       [ 9.64183076, -0.16085942,  0.26112048],
       [ 8.00085627,  3.64601062,  1.39057924],
       [ 0.61593501, 12.74309014,  3.65357209],
       [-1.58130543,  1.7053092 ,  7.58560229],
       [ 9.21446526,  6.06476372, -0.28810469],
       [-1.47398961,  8.88599493, 10.67290195],
       [12.41473338,  0.22993611, -0.13660595],
       [42.36182432,  0.50963947, -0.9632997 ],
       [10.62635356,  1.39183591,  3.44946619],
       [ 3.68611828, -0.75562734, -1.55419767]])

In [115]:
np.exp(-r*Ts_)

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [107]:
rdm.randn(len(Ts),nSim)

array([[ 1.95519129, -0.54687235,  0.61372968],
       [ 0.5443173 ,  0.78402711,  0.78406931],
       [-2.36306516, -0.62724119,  1.8843635 ],
       [ 0.48505782,  0.51801641, -0.70824372],
       [ 1.08214226, -0.41619529,  0.31654051],
       [ 0.62455522,  1.1277306 , -0.14014768],
       [-1.19619012,  0.16769753,  0.66425694],
       [ 0.36185559, -1.5571294 , -0.56541126],
       [-0.02764433, -0.09576143,  1.26058567],
       [-0.91602125,  0.33918933,  0.62739784],
       [ 0.61228389,  0.73806925,  0.37791967]])

In [9]:
g1 = lambda xT,Delta: Delta["_"] + Delta["t"] * T \
                    + Delta["x"] * (xT - x0) + Delta["xx"] * (xT - x0)**2/2
# Stock Grid
xMin,xMax = 0., 2*K
xs     = np.linspace(xMin,xMax,1001)
# Max Bachelier volatility (normalized)
sigMax = 3.5
# Target prices
pBS = bsVanilla(xs,K,r,delta,sig,T)

def update_Price(message):
    if message['name'] == 'value':
        sig_         = slider.value
        G1,p1,Delta  = statRep(G,sigBa = sig_ * x0)
        plot.y       = g1(xs,Delta)
        figure.title = title(sig_)

#===================== PLOT =====================# 
title = lambda sig: 'Black-Scholes Price, K = %2.f, T = %2.f, volatility = %2.f%%'%(K,T,100*sig)

figure = bqplt.figure(title = title(0.),animation_duration=0,legend_location = "top-left")
bqplt.xlabel("Stock Price")

# Plot
bqplt.plot(xs,bsVanilla(xs,K,r,delta,0.,T)[:2],stroke_width = 3,
           colors = ["steelblue","purple"],opacities = [0.4],line_style = "dashed")

plot = bqplt.plot(xs,bsVanilla(xs,K,r,delta,0.,T)[:2],stroke_width = 2,
                  labels = ["Call","Put"],display_legend = True, colors = ["steelblue","purple"],
                  axes_options={"x": {"label": "Stock Price", "num_ticks":21}})

s = K*np.exp(-r*T)
bqplt.plot([s,s],[-5.,xMax+10.],stroke_width = 2,colors = ["orchid"],opacities = [0.6],line_style = "solid")
bqplt.plot([K,K],[-5.,xMax+10.],stroke_width = 2,colors = ["gray"],opacities = [0.8],line_style = "solid")

plot.observe(update_Price, ['x','y'])
bqplt.xlim(xMin,xMax); bqplt.ylim(0.,xMax)# - K)
# Slider (log2 of number of time steps)
slider = wi.FloatSlider(description=r'$\sigma$', step=0.01, min = 0, max = sigMax)
slider.observe(update_Price,'value')
# Display animation
wi.VBox([slider, figure])

NameError: name 'sig' is not defined